In [1]:
import pandas as pd

In [2]:
# The path to our CSV file
file = "Resources/population_data.csv"

population_df = pd.read_csv(file, encoding='utf-8')


population_df.head()

,cca2,name,pop2020,pop2019,pop2050,pop2030,pop2015,pop2010,pop2000,pop1990,pop1980,pop1970,area,Density,GrowthRate,WorldPercentage,rank
0,CN,China,1439323.776,1433783.686,1402405.170,1464340.159,1406847.870,1368810.615,1290550.765,1176883.674,1000089.235,827601.394,9706961,148.2775,1.0039,0.1847,1
1,IN,India,1380004.385,1366417.754,1639176.033,1503642.322,1310152.403,1234281.170,1056575.549,873277.798,698952.844,555189.792,3287590,419.7617,1.0099,0.1770,2
2,US,United States,331002.651,329064.917,379419.102,349641.876,320878.310,309011.475,281710.909,252120.309,229476.354,209513.341,9372610,35.3160,1.0059,0.0425,3
3,ID,Indonesia,273523.615,270625.568,330904.664,299198.430,258383.256,241834.215,211513.823,181413.402,147447.836,114793.178,1904569,143.6144,1.0107,0.0351,4
4,PK,Pakistan,220892.340,216565.318,338013.196,262958.794,199426.964,179424.641,142343.578,107647.921,78054.343,58142.060,881912,250.4698,1.0200,0.0283,5


In [3]:
new_population_df = population_df[['name','pop2019','area', 'Density', 'GrowthRate', 'WorldPercentage']]
new_population_df

,name,pop2019,area,Density,GrowthRate,WorldPercentage
0,China,1433783.686,9706961,148.2775,1.0039,0.1847
1,India,1366417.754,3287590,419.7617,1.0099,0.1770
2,United States,329064.917,9372610,35.3160,1.0059,0.0425
3,Indonesia,270625.568,1904569,143.6144,1.0107,0.0351
4,Pakistan,216565.318,881912,250.4698,1.0200,0.0283
...,...,...,...,...,...,...
227,Montserrat,4.989,102,48.9412,1.0006,0.0000
228,Falkland Islands,3.377,12173,0.2859,1.0305,0.0000
229,Niue,1.615,260,6.2538,1.0068,0.0000
230,Tokelau,1.340,12,113.0833,1.0127,0.0000


In [4]:
renamed_population_df = new_population_df.rename(columns={'name':'country', 'pop2019':'population','area':'area','Density':'density', 'GrowthRate':'population_growth_rate', 'WorldPercentage':'world_percentage'})
renamed_population_df

,country,population,area,density,population_growth_rate,world_percentage
0,China,1433783.686,9706961,148.2775,1.0039,0.1847
1,India,1366417.754,3287590,419.7617,1.0099,0.1770
2,United States,329064.917,9372610,35.3160,1.0059,0.0425
3,Indonesia,270625.568,1904569,143.6144,1.0107,0.0351
4,Pakistan,216565.318,881912,250.4698,1.0200,0.0283
...,...,...,...,...,...,...
227,Montserrat,4.989,102,48.9412,1.0006,0.0000
228,Falkland Islands,3.377,12173,0.2859,1.0305,0.0000
229,Niue,1.615,260,6.2538,1.0068,0.0000
230,Tokelau,1.340,12,113.0833,1.0127,0.0000


In [5]:
renamed_population_df = renamed_population_df.set_index("country")
renamed_population_df.head()

,population,area,density,population_growth_rate,world_percentage
country,,,,,
China,1433783.686,9706961,148.2775,1.0039,0.1847
India,1366417.754,3287590,419.7617,1.0099,0.1770
United States,329064.917,9372610,35.3160,1.0059,0.0425
Indonesia,270625.568,1904569,143.6144,1.0107,0.0351
Pakistan,216565.318,881912,250.4698,1.0200,0.0283


In [ ]:
# The path to our CSV file
file = "Resources/happy_2019.csv"

hpi_country_df = pd.read_csv(file, encoding='UTF-8')
new_hpi_country_df = hpi_country_df.rename(columns={'Country or region':'country'})

hpi_final = new_hpi_country_df['country']
hpi_final.head(155)

In [ ]:
merge_df = pd.merge(renamed_population_df, hpi_final, on=('country'), how='inner')

clean_df = merge_df.rename(columns={ 'sub_region':'region'})

clean_df

In [6]:
from sqlalchemy import create_engine
from config import postgres_pwd

db_name = "happiness_db"
table_name = "population"

connection_string = f"postgres:{postgres_pwd}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')
conn = engine.connect()

clean_df.to_sql(table_name, con=engine, if_exists="append", index=False)

conn.close()